# Stoneburner, Kurt
- ## DSC 530 - Week XX
- ## Chapter X, Exercise X


In [2]:
# //****************************************************************************************
# //*** Set Working Directory to thinkstats folder.
# //*** This pseudo-relative path call should work on all Stoneburner localized projects. 
# //****************************************************************************************

import os
import sys
workingPath = os.getcwd().replace("coding", "ThinkStats2\\code")
sys.path.insert(1, workingPath)
os.chdir(workingPath)

In [35]:
# //*** Imports and Load Data
import nsfg
import thinkstats2
import thinkplot
import first
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import scipy

#resp = nsfg.ReadFemResp()
#preg = nsfg.ReadFemPreg()


In [4]:
# //*** CODE HERE

- ## Chapter 11, Exercise 1

**Exercise:** Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.

In [5]:
import first
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]

In [48]:
#print(f"{np.corrcoef(live.columns)}")
cor_val = []
index = []
for col in live.columns:
    loop_val = np.corrcoef(live['prglngth'],live[col])[0][1]
    if math.isnan(loop_val) == False:
        cor_val.append(loop_val)
        index.append(col)

cors = pd.Series(index=index, data=cor_val).sort_values()
cor_pos = cors [ cors > 0 ]
neg_cors = cors [ cors < 0] 
print(f"{cor_pos}")
print(f"{neg_cors}")
#print(f"{[ _ for _ in cors.index ]}")
#print(f"{live}")
#print(f"{live[isinstance(live.dtypes,int) == True ]}")




    

bfeedwks_i         0.000548
hisprace           0.001750
secu_p             0.002620
agescrn            0.003323
ager               0.003323
insuranc           0.003351
paydeliv_i         0.004408
brnout             0.005201
caseid             0.006111
anyusint           0.006181
insuranc_i         0.007210
hieduc_i           0.007636
basewgt            0.009253
adj_mod_basewgt    0.010529
pmarpreg           0.010683
pubassis           0.011673
finalwgt           0.012495
religion           0.014954
metro              0.017178
race               0.017731
hieduc             0.020130
educat             0.024304
hispanic           0.025692
gestasun_w         0.032425
poverty            0.033517
sest               0.036364
gestasun_m         0.040708
pregend1           0.047428
mosgest            0.309232
lbw1               0.322064
prglngth           1.000000
dtype: float64
prglngth_i      -0.148505
cmlastlb        -0.045201
fmarcon5_i      -0.044364
cmlstprg        -0.035817
birthord     

The following are the only variables I found that have a statistically significant effect on pregnancy length.

In [7]:
import statsmodels.formula.api as smf
model = smf.ols('prglngth ~ birthord==1 + race==2 + nbrnaliv>1', data=live)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     34.28
Date:                Mon, 26 Oct 2020   Prob (F-statistic):           5.09e-22
Time:                        12:04:06   Log-Likelihood:                -18247.
No. Observations:                8884   AIC:                         3.650e+04
Df Residuals:                    8880   BIC:                         3.653e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                38.7617      0.039   1006.410      0.000      38.686      38.837
birthord == 1[T.True]     0.1015      0.040      2.528      0.011       0.023       0.180
race == 2[T.True]         0.1390      0.042      3.311      0.001       0.057       0.221
nbrnaliv > 1[T.True]     -1.4944      0.164     -9.086      0.000      -1.817      -1.172
==============================================================================
Omnibus:                     1587.470   Durbin-Watson:                   1.619
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6160.751
Skew:                          -0.852   Prob(JB):                         0.00
Kurtosis:                       6.707   Cond. No.                         10.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- ## Chapter 11, Exercise 2

**Exercise:** The Trivers-Willard hypothesis suggests that for many mammals the sex ratio depends on “maternal condition”; that is, factors like the mother’s age, size, health, and social status. See https://en.wikipedia.org/wiki/Trivers-Willard_hypothesis

Some studies have shown this effect among humans, but results are mixed. In this chapter we tested some variables related to these factors, but didn’t find any with a statistically significant effect on sex ratio.

As an exercise, use a data mining approach to test the other variables in the pregnancy and respondent files. Can you find any factors with a substantial effect?



In [8]:
import regression
join = regression.JoinFemResp(live)

- ## Chapter 11, Exercise 3

**Exercise:** If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called `poisson`. It works the same way as `ols` and `logit`. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called `numbabes`.

Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?



In [9]:
# //*** CODE HERE

Now we can predict the number of children for a woman who is 35 years old, black, and a college
graduate whose annual household income exceeds $75,000